In [14]:
import numpy as np
import cv2

# Dilation
def dilate_(src, kernel, iterations=1):
    img = src.copy()
    for iters in range(iterations):
        img = np.pad(img, ((1, 1), (1, 1)), "constant", constant_values = ((0, 0), (0, 0)))  # padding ,pad 0
        dilation_output = np.zeros(img.shape, dtype=np.uint8)  # blank space
        
        #! specify the range for rows and columns
        for center_row in range(1, img.shape[0]-1):
            for center_col in range(1, img.shape[1]-1):
                #! get patch (3 x 3)
                x = img[center_row-1:center_row+2, center_col-1:center_col+2]
                #! get mul (elementwise-mutiply)
                mul = np.multiply(x, kernel)
                #! condition : if mul have any 255,than set center-pixel = 255, other = 0
                if mul.sum() > 0:
                    dilation_output[center_row, center_col] = 255
                else:
                    dilation_output[center_row, center_col] = 0

        img = dilation_output[1:-1, 1:-1]
    return img

# Erosion
def erode_(src, kernel, iterations=1):
    img = src.copy()
    for iters in range(iterations):
        img = np.pad(img, ((1, 1), (1, 1)), "constant", constant_values = ((0, 0), (0, 0)))  # padding,pad 0
        erosion_output = np.zeros(img.shape, dtype=np.uint8)  # blank space
        
        #! specify the range for rows and columns
        for center_row in range(1, img.shape[0]-1):
            for center_col in range(1, img.shape[1]-1):
                #! get patch (3 x 3)
                x = img[center_row-1:center_row+2, center_col-1:center_col+2]
                #! get mul (elementwise-mutiply)
                mul = np.multiply(x, kernel)
                #! condition : if mul and 255*kernel are equal, than center pixel = 255, other = 0
                if np.array_equal(mul, np.multiply(kernel, 255)):
                    erosion_output[center_row, center_col] = 255       
                else:
                    erosion_output[center_row, center_col] = 0            

        img = erosion_output[1:-1, 1:-1]
    return img

# Opening
def open_(src, kernel, iterations=1):
    #! complete the opening operation
    img = erode_(src, kernel, iterations=2)
    img = dilate_(img, kernel, iterations=2)
    
    return img 
# Closing
def close_(src, kernel, iterations=1):
    #! complete the closing operation
    img = dilate_(src, kernel, iterations=2)
    img = erode_(img, kernel, iterations=2)

    return img
# for display
def put_label(img, text):
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2.putText(img_color, text, (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        return img_color

if __name__ == '__main__':
    # read image
    source_img = cv2.imread("j.png", 0)    
    open_img = cv2.imread("opening_j.png", 0)    
    close_img = cv2.imread("closing_j.png", 0)

    # set kernel
    kernel = np.array([
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]
    ], dtype=np.uint8)

    # your version
    dilation = dilate_(source_img, kernel, iterations=3)
    erosion = erode_(source_img, kernel, iterations=3)
    opening = open_(open_img, kernel, iterations=2)
    closing = close_(close_img, kernel, iterations=2)

    # opencv version anwser
    dilation_ans =  cv2.dilate(source_img, kernel, iterations=3)
    erosion_ans = cv2.erode(source_img, kernel, iterations=3)
    opening_ans = cv2.morphologyEx(open_img, cv2.MORPH_OPEN, kernel, iterations=2)
    closing_ans = cv2.morphologyEx(close_img, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # check answer
    print("dilate Check:\t",np.array_equal(dilation, dilation_ans))
    print("erosion Check:\t",np.array_equal(erosion, erosion_ans))
    print("opening Check:\t",np.array_equal(opening, opening_ans))
    print("closing Check:\t",np.array_equal(closing, closing_ans))

    # display
    d1 = put_label(dilation, "My Dilation")
    d2 = put_label(dilation_ans, "OpenCV")
    e1 = put_label(erosion, "My Erosion")
    e2 = put_label(erosion_ans, "OpenCV")
    o1 = put_label(opening, "My Opening")
    o2 = put_label(opening_ans, "OpenCV")
    c1 = put_label(closing, "My Closing")
    c2 = put_label(closing_ans, "OpenCV")

    input_src = put_label(source_img, "src")
    input_opening = put_label(open_img, "scr")
    input_closing = put_label(close_img, "scr")

    row1 = np.hstack((input_src, input_src, input_opening, input_closing))
    row2 = np.hstack((d1, e1, o1, c1))
    row3 = np.hstack((d2, e2, o2, c2))
    combined = np.vstack((row1, row2, row3))

    cv2.imshow("All Results Comparison", combined)
    cv2.imwrite("result_compare.png", combined)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

dilate Check:	 True
erosion Check:	 True
opening Check:	 True
closing Check:	 True
